<a href="https://colab.research.google.com/github/alexmal2804/hwds/blob/master/course_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Описание входного DataSet
https://www.icpsr.umich.edu/web/NACJD/studies/2581/versions/V1#

Database for Forensic Anthropology in the United States, 1962-1991 (ICPSR 2581)

Этот проект был осуществлен с целью создания компьютеризированной скелетной базы данных, состоящей из недавних судебно-медицинских дел, чтобы отразить нынешнее этническое разнообразие и демографическую структуру населения Соединенных Штатов. Цель состояла в том, чтобы накопить судебно-судебно-скелетную выборку, достаточно разнообразную, чтобы отразить различные социально-экономические группы населения в целом из разных географических регионов страны, чтобы исследователи могли разработать стандарты, используемые для скелетной идентификации. 

In [ ]:
# Импорт необходимых для работы модулей
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression as LR
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.model_selection import train_test_split
import matplotlib
from matplotlib import pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

##Загружаем DataSet
Используемый набор данных на ресурсе был представлен в виде 7  DataSet в формате "stata". Данные в датасетах были связаны первичным ключом 'FDN'. Необходимые для анализа данные были предварительно объединены в R-Studio и импортированы в csv.

In [31]:
# загружаем csv с данными в colab
from google.colab import files
uploaded = files.upload()

Saving res_data_1.csv to res_data_1.csv


In [ ]:
 # df = pd.read_csv('https://github.com/alexmal2804/hwds/blob/master/res_data_1.csv', delimiter=';')

In [46]:
 # Импортируем в DataFrame
 df = pd.read_csv('/content/res_data_1.csv', delimiter=';')

In [47]:
df.head()

,FDN,CONTRIB,CURATOR,RECORDER,DATEREC,IDSTAT,DATPOSID,SEX,SEXSTAT,RACE,RACESTAT,ETHNIC,DOB_MODA,DOB_YR,DEATHAGE,LOWERAGE,UPPERAGE,AGEGROUP,FSTATURE,CSTATURE,FWEIGHT,CWEIGHT,GOL,BNL,BBH,XCB,WFB,ZYB,AUB,BPL,NLH,NLB,MAB,MAL,MDH,OBH,OBB,DKB,EKB,FRC,...,ISCHLN_L,ISCHLN_R,FEMXLN_L,FEMXLN_R,FEMBLN_L,FEMBLN_R,FEMEBR_L,FEMEBR_R,FEMHDD_L,FEMHDD_R,FEMSAP_L,FEMSAP_R,FEMSTV_L,FEMSTV_R,FEMMAP_L,FEMMAP_R,FEMMTV_L,FEMMTV_R,FEMCIR_L,FEMCIR_R,TIBXLN_L,TIBXLN_R,TIBPEB_L,TIBPEB_R,TIBDEB_L,TIBDEB_R,TIBNFX_L,TIBNFX_R,TIBNFT_L,TIBNFT_R,TIBCIR_L,TIBCIR_R,FIBXLN_L,FIBXLN_R,FIBMDM_L,FIBMDM_R,CALCXL_L,CALCXL_R,CALCBR_L,CALCBR_R
0,1,LA84-7,LSU,BLANK,06.16.1986,1.0,5.30.84,F,1,B,1,NaN,BLANK,1951,33,33,33,A,165,999,134,999,171.0,92.0,122.0,130.0,92.0,117.0,112.0,96.0,50.0,25.0,65.0,56.0,33.0,34.0,40.0,21.0,92.0,102.0,...,77.0,NaN,467.0,NaN,462.0,NaN,72.0,NaN,40.0,NaN,24.0,NaN,26.0,NaN,28.0,NaN,24.0,NaN,80.0,NaN,392.0,NaN,68.0,NaN,47.0,NaN,32.0,NaN,20.0,NaN,82.0,NaN,383.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN
1,2,LA85-7,LSU,BLANK,06.13.1986,3.0,NaN,M,3,B,3,NaN,BLANK,1955,99,25,32,A,185,999,175,999,185.0,NaN,NaN,NaN,92.0,122.0,NaN,NaN,50.0,22.0,NaN,NaN,32.0,32.0,38.0,NaN,96.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,81.0,81.0,45.0,NaN,30.0,29.0,31.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,422.0,418.0,NaN,16.0,84.0,85.0,47.0,46.0
2,3,LA84-10,LSU,BLANK,06.14.1986,2.0,NaN,M,2,W,2,NaN,NaN,1947*,99,30,37,A,999,999,999,999,172.0,97.0,137.0,138.0,93.0,127.0,122.0,94.0,51.0,25.0,59.0,51.0,34.0,35.0,39.0,21.0,94.0,105.0,...,91.0,92.0,453.0,454.0,450.0,453.0,88.0,89.0,49.0,51.0,30.0,30.0,33.0,33.0,30.0,31.0,28.0,28.0,90.0,91.0,366.0,366.0,82.0,82.0,57.0,55.0,36.0,36.0,26.0,26.0,98.0,95.0,360.0,360.0,17.0,16.0,91.0,91.0,46.0,46.0
3,4,LA84-12,LSU,BLANK,06.13.1986,1.0,11.??.84,F,1,W,1,NaN,BLANK,1945,35,35,35,A,168,999,142,999,185.0,107.0,143.0,NaN,101.0,122.0,119.0,97.0,50.0,NaN,57.0,53.0,31.0,36.0,42.0,24.0,NaN,117.0,...,NaN,NaN,435.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,342.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,LA85-1,LSU,BLANK,06.13.1986,2.0,NaN,F,1,B,3,NaN,NaN,1952*,99,25,33,A,999,999,999,999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,82.0,83.0,410.0,NaN,402.0,NaN,77.0,77.0,43.0,42.0,26.0,26.0,25.0,25.0,26.0,NaN,21.0,NaN,73.0,NaN,348.0,346.0,75.0,75.0,46.0,46.0,29.0,28.0,21.0,20.0,78.0,76.0,344.0,342.0,14.0,15.0,79.0,82.0,38.0,39.0


In [48]:
# размер DataFrame
df.shape

(1514, 142)

##Постановка задачи
Необходимо обучить модель используя алгоритм логистической регрессии  определить по данным краниальных и посткраниальных измерений пол, рассовую принадлежность и возрастную группу человека.

Т.к. нам нужны не все столбцы в представленном DataSet, а только результаты измерений на которых будем обучать модель и категориальные столбцы со значением целевой функции - удалим те, которые не будут использоваться:

In [49]:
# Удаляем неиспользуемые столбцы оставля только те, которые будут использоваться в моделях
df = df.drop(columns=['FDN', 'CONTRIB', 'CURATOR', 'RECORDER', 'DATEREC', 'IDSTAT', 'SEXSTAT', 'RACESTAT', 'ETHNIC', 'DOB_MODA', 'DOB_YR', 'DEATHAGE', 'LOWERAGE', 'UPPERAGE', 'FSTATURE', 'CSTATURE', 'FWEIGHT', 'CWEIGHT','DATPOSID'])
df.head()

,SEX,RACE,AGEGROUP,GOL,BNL,BBH,XCB,WFB,ZYB,AUB,BPL,NLH,NLB,MAB,MAL,MDH,OBH,OBB,DKB,EKB,FRC,PAC,OCC,FOL,FOB,GNI,HML,TML,GOG,CDL,WRL,MRL,XRL,MLT,MAN,UFHT,UFBR,CLAXLN_L,CLAXLN_R,CLAAPD_L,...,ISCHLN_L,ISCHLN_R,FEMXLN_L,FEMXLN_R,FEMBLN_L,FEMBLN_R,FEMEBR_L,FEMEBR_R,FEMHDD_L,FEMHDD_R,FEMSAP_L,FEMSAP_R,FEMSTV_L,FEMSTV_R,FEMMAP_L,FEMMAP_R,FEMMTV_L,FEMMTV_R,FEMCIR_L,FEMCIR_R,TIBXLN_L,TIBXLN_R,TIBPEB_L,TIBPEB_R,TIBDEB_L,TIBDEB_R,TIBNFX_L,TIBNFX_R,TIBNFT_L,TIBNFT_R,TIBCIR_L,TIBCIR_R,FIBXLN_L,FIBXLN_R,FIBMDM_L,FIBMDM_R,CALCXL_L,CALCXL_R,CALCBR_L,CALCBR_R
0,F,B,A,171.0,92.0,122.0,130.0,92.0,117.0,112.0,96.0,50.0,25.0,65.0,56.0,33.0,34.0,40.0,21.0,92.0,102.0,110.0,85.0,38.0,28.0,37.0,31.0,8.0,92.0,NaN,30.0,NaN,NaN,79.0,130.0,69.0,97.0,NaN,140.0,NaN,...,77.0,NaN,467.0,NaN,462.0,NaN,72.0,NaN,40.0,NaN,24.0,NaN,26.0,NaN,28.0,NaN,24.0,NaN,80.0,NaN,392.0,NaN,68.0,NaN,47.0,NaN,32.0,NaN,20.0,NaN,82.0,NaN,383.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN
1,M,B,A,185.0,NaN,NaN,NaN,92.0,122.0,NaN,NaN,50.0,22.0,NaN,NaN,32.0,32.0,38.0,NaN,96.0,NaN,114.0,NaN,NaN,NaN,NaN,29.0,10.0,94.0,113.0,32.0,NaN,62.0,84.0,125.0,69.0,102.0,153.0,160.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,81.0,81.0,45.0,NaN,30.0,29.0,31.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,422.0,418.0,NaN,16.0,84.0,85.0,47.0,46.0
2,M,W,A,172.0,97.0,137.0,138.0,93.0,127.0,122.0,94.0,51.0,25.0,59.0,51.0,34.0,35.0,39.0,21.0,94.0,105.0,107.0,108.0,36.0,32.0,31.0,29.0,11.0,97.0,121.0,30.0,NaN,60.0,74.0,127.0,68.0,NaN,153.0,144.0,12.0,...,91.0,92.0,453.0,454.0,450.0,453.0,88.0,89.0,49.0,51.0,30.0,30.0,33.0,33.0,30.0,31.0,28.0,28.0,90.0,91.0,366.0,366.0,82.0,82.0,57.0,55.0,36.0,36.0,26.0,26.0,98.0,95.0,360.0,360.0,17.0,16.0,91.0,91.0,46.0,46.0
3,F,W,A,185.0,107.0,143.0,NaN,101.0,122.0,119.0,97.0,50.0,NaN,57.0,53.0,31.0,36.0,42.0,24.0,NaN,117.0,113.0,100.0,33.0,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0,NaN,NaN,NaN,NaN,...,NaN,NaN,435.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,342.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,F,B,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.0,131.0,12.0,...,82.0,83.0,410.0,NaN,402.0,NaN,77.0,77.0,43.0,42.0,26.0,26.0,25.0,25.0,26.0,NaN,21.0,NaN,73.0,NaN,348.0,346.0,75.0,75.0,46.0,46.0,29.0,28.0,21.0,20.0,78.0,76.0,344.0,342.0,14.0,15.0,79.0,82.0,38.0,39.0
